# Programme de Matching

## Preparation fichier

### Variable Ville

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex
    
### Step 1: Match uniquement les 1 dans INSEE/INPI

Les siren qui ont uniquement un seul établissement a l'INSEE et à l'INPI.

### Step 2: match Ville/CP/CC

### Step 3: Match adresse

## Description Algorithm

## Description Input

## Description Output

In [1]:
from inpi_insee import siretisation

%load_ext autoreload
%autoreload 2


C:\Users\PERNETTH\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
param = {
    'communes_insee': r'data\input\communes_france.csv',
    'upper_word':r'data\input\upper_stop.csv',
    'insee': r'data\input\insee_2017_7480120.gz',
    'inpi_etb': r'data\input\inpi_etb_4979272.gz',
}

In [3]:
al_siret = siretisation.siretisation_inpi(param)

## Preparation adresse

In [ ]:
test_adress = al_siret.clean_commune()

In [ ]:
test_adress.to_csv('data\input\inpi_etb_cleaned_{}.gz'.format(
    test_adress.shape[0]),
                        compression='gzip', index = False) 

## Match 1-1

In [ ]:
inpi = r'data\input\inpi_etb_cleaned_4979272.gz'
al_siret.match_unique_etb(inpi)

## Merge ville/code postale/ code commune

In [ ]:
%%time
import pandas as pd

inpi_dtype = {
    'siren': 'object',
    'index': 'object',
 'Type': 'object',
 'Adresse_Ligne1': 'object',
 'Adresse_Ligne2': 'object',
 'Adresse_Ligne3': 'object',
 'Code_Postal': 'object',
 'Ville': 'object',
 'Code_Commune': 'object',
 'Pays': 'object',
 'count_initial_inpi': 'int',
 'ncc': 'object',
}

inpi_col =['siren',
        'index',
 'Type',
 'Adresse_Ligne1',
 'Adresse_Ligne2',
 'Adresse_Ligne3',
 'Code_Postal',
 'Ville',
 'Code_Commune',
 'Pays',
 'count_initial_inpi',
 'ncc']

df_ets = r'data\input\unmatched\unmatch_1_unique_siren_1969368.gz'
inpi = al_siret.import_dask(file=df_ets,
                       usecols=inpi_col, dtype=inpi_dtype)

df_nomatch = pd.DataFrame()
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                  option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='1_ville_cp_cc',
        to_csv=True)

### Creation address

In [ ]:
inpi_dtype = {
    'siren': 'object',
    'index': 'object',
 'Type': 'object',
 'Adresse_Ligne1': 'object',
 'Adresse_Ligne2': 'object',
 'Adresse_Ligne3': 'object',
 'Code_Postal': 'object',
 'Ville': 'object',
 'Code_Commune': 'object',
 'Pays': 'object',
 'count_initial_inpi': 'int',
 'ncc': 'object',
}

inpi_col =['siren',
        'index',
 'Type',
 'Adresse_Ligne1',
 'Adresse_Ligne2',
 'Adresse_Ligne3',
 'Code_Postal',
 'Ville',
 'Code_Commune',
 'Pays',
 'count_initial_inpi',
 'ncc']
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_1_ville_cp_cc_744141.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
al_siret.prepare_adress(inpi)

### Loop

In [4]:
%%time
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


#df_nomatch = pd.DataFrame()
df_match = pd.DataFrame()
for chunk in [0,1
              ,2,3,4,5,6,7,8,9, 10,11,12
             ]:
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(chunk)
    inpi = al_siret.import_dask(file=csv_file,
                       usecols=inpi_col, dtype=inpi_dtype)
    df_input = inpi
    total_match = pd.DataFrame()
    for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
        df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                      option=i,
                                      regex_go = True)

        df_input = df_input_['unmatch']
        total_match = total_match.append(df_input_['true_match']) 
    #df_nomatch = df_nomatch.append(df_input)
    df_match = df_match.append(total_match)

[########################################] | 100% Completed |  1min  0.4s
[########################################] | 100% Completed | 47.4s
[########################################] | 100% Completed | 49.0s
[########################################] | 100% Completed | 52.7s
[########################################] | 100% Completed | 44.1s
[########################################] | 100% Completed | 49.6s
[########################################] | 100% Completed | 49.5s
[########################################] | 100% Completed | 47.2s
[########################################] | 100% Completed | 50.2s
[########################################] | 100% Completed | 53.7s
[########################################] | 100% Completed | 43.0s
[########################################] | 100% Completed | 50.8s
[########################################] | 100% Completed | 53.0s
[########################################] | 100% Completed | 44.1s
[########################################]

In [5]:
df_temp = pd.DataFrame()
for i in range(0,13):
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(i)
    df_temp = df_temp.append(pd.read_csv(csv_file,
                                         dtype= inpi_dtype)
                            )

In [9]:
299986 / 4979272

0.0602469597965325

In [7]:
al_siret.match_unmatch(
        df_inpi_initial=df_temp,
        df_inpi_mergeboth=df_match.drop(columns = '_merge'),
        step='3_adress',
        to_csv=True)

,siren,index,Type,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,Code_Postal,Ville,Code_Commune,Pays,ncc,count_initial_inpi,Adress_new,Adresse_new_clean_reg
444155,388239667,32,PRI,NaN,38 Chemin du Moulin Carron,NaN,69130,Écully,69081,France,ECULLY,2,38 Chemin du Moulin Carron,MOULIN$|CARRON$
444156,814837621,33,SEP,NaN,17 rue Centrale,NaN,01200,Bellegarde-sur-Valserine,01033,FRANCE,BELLEGARDE SUR VALSERINE,1,17 rue Centrale,CENTRALE$
444157,331221150,39,SEP,NaN,231 avenue de Parme,Inopolis Bat C,01000,Bourg-en-Bresse,01053,FRANCE,BOURG EN BRESSE,5,231 avenue de Parme Inopolis Bat C,PARME$|INOPOLIS$|BAT$
444158,306168568,162,SEP,NaN,Zone d'Activité Macon Est,NaN,01750,Replonges,01320,FRANCE,REPLONGES,1,Zone d'Activité Macon Est,ZONE$|ACTIVITE$|MACON$
444159,331455691,168,SEC,NaN,17 Rue Joseph Marie Jacquard,NaN,69680,Chassieu,69271,France,CHASSIEU,7,17 Rue Joseph Marie Jacquard,JOSEPH$|MARIE$|JACQUARD$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744136,483625141,4976428,SEC,Zac l'Hay Parc,NaN,NaN,94240,L'Hay-les-Roses,94038,FRANCE,NaN,18,Zac l'Hay Parc,ZAC$|HAY$|PARC$
744137,562118349,4976573,PRI,63 Boulevard de la Marne,NaN,NaN,94210,La Varenne Saint-Hilaire,94068,FRANCE,NaN,1,63 Boulevard de la Marne,MARNE$
744138,510324981,4976813,PRI,NaN,Zone Juliette Bâtiment 124 C,NaN,94551,Orly Aérogare CEDEX,94900,FRANCE,NaN,1,Zone Juliette Bâtiment 124 C,ZONE$|JULIETTE$|BATIMENT$
744139,303814511,4977200,SEC,39 Rue DE LA BRESSE,NaN,NaN,94150,MIN RUNGIS,94065,FRANCE,NaN,2,39 Rue DE LA BRESSE,BRESSE$


### Matching voie

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_3_adress_299986.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [13]:
temp = al_siret.merge_siren_candidat(df_input=inpi,
                              regex_go=True, 
                              matching_voie=True,
                              option=['ncc', 'libelleCommuneEtablissement'])

[########################################] | 100% Completed | 36.5s
[########################################] | 100% Completed | 36.5s
[########################################] | 100% Completed | 36.5s


In [14]:
temp.head()

,siren,index,Type,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,Code_Postal,Ville,Code_Commune,Pays,...,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,count_initial_insee,_merge,siret_test1,digit_inpi,test
13,552120222,120763,SEC,NaN,15 place du Général de Gaulle,NaN,50230,Agôn Coutainville,NaN,FRANCE,...,50003,NaN,NaN,NaN,NaN,3205.0,both,55212022233386,15,True
96,352297360,131950,SEP,NaN,285 rue du Villard,NaN,01380,Saint-André-de-Bage,01332,FRANCE,...,01332,NaN,NaN,NaN,NaN,2.0,both,35229736000021,285,True
152,305307274,141530,PRI,10 rue Albert Ledant,NaN,NaN,02580,Etréaupont,02295,France,...,02295,NaN,NaN,NaN,NaN,3.0,both,30530727400038,10,True
155,378844328,142678,PRI,17 avenue du General de Gaulle,NaN,NaN,02580,Etréaupont,02295,France,...,02295,NaN,NaN,NaN,NaN,2.0,both,37884432800013,17,True
342,379853567,183554,SEP,NaN,12 route de Bellerive,NaN,03110,Cognât Lyonne,03080,FRANCE,...,03080,NaN,NaN,NaN,NaN,3.0,both,37985356700020,12,True


In [ ]:
temp = (temp
                  .merge(
                      (temp
                       .groupby(['siren', option[1]])['siren']
                       .count()
                       .rename('count')
                       .reset_index()
                       )
                  )
                  )

In [ ]:
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_3_adress_299986.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='1_ville_cp_cc',
        to_csv=True)